In [1]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import datetime as dt
from sklearn import preprocessing

In [2]:
data = pd.read_csv('accre-gpu-jobs-2022-v2.csv', on_bad_lines='skip')

In [3]:
data

,JOBID,ACCOUNT,USER,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,GPUS,PARTITION,EXITCODE,STATE
0,32880657,malanga,arline,18.66M,2-00:00:00,00:13:13,1,2,1,pascal,0:0,COMPLETED
1,32880701,glasshouse,brady,0,05:00:00,00:00:22,1,3,1,maxwell,0:0,COMPLETED
2,32880718,glasshouse,brady,402.32M,05:00:00,00:08:57,1,3,1,maxwell,0:0,COMPLETED
3,32880719,glasshouse,brady,417.62M,20:00:00,05:09:46,1,1,1,pascal,0:0,COMPLETED
4,32880720,glasshouse,brady,450.09M,15:00:00,04:11:10,1,1,1,pascal,0:0,COMPLETED
...,...,...,...,...,...,...,...,...,...,...,...,...
336945,46754520_26,harovin,porsha,0,5-00:00:00,00:07:07,1,6,1,turing,0:0,RUNNING
336946,46754542,malanga,arline,0,2-00:00:00,00:00:00,0,2,1,pascal,0:0,PENDING
336947,46754560,malanga,arline,0,2-00:00:00,00:00:00,0,2,1,pascal,0:0,PENDING
336948,46754598,glasshouse,treva,145.24M,1-00:00:00,00:01:26,1,5,1,turing,0:0,COMPLETED


In [4]:
data = (data[
    (data['STATE']=='COMPLETED') & 
    (data['USEDMEM']!='0') &
    (data['USEDMEM']!='0.00M')])

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 277475 entries, 0 to 336948
Data columns (total 12 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   JOBID      277475 non-null  object
 1   ACCOUNT    277475 non-null  object
 2   USER       277475 non-null  object
 3   USEDMEM    277475 non-null  object
 4   REQTIME    277475 non-null  object
 5   USEDTIME   277475 non-null  object
 6   NODES      277475 non-null  int64 
 7   CPUS       277475 non-null  int64 
 8   GPUS       277475 non-null  int64 
 9   PARTITION  277475 non-null  object
 10  EXITCODE   277475 non-null  object
 11  STATE      277475 non-null  object
dtypes: int64(3), object(9)
memory usage: 27.5+ MB


In [6]:
def get_seconds(time_str):
    if len(time_str) == 8:
        hh, mm, ss = time_str.split(':')
        return int(hh) * 3600 + int(mm) * 60 + int(ss)
    else: 
        d = time_str[0]
        dropday = time_str[2:]
        hh, mm, ss = dropday.split(':')
        return int(d)*3600*24 + int(hh) * 3600 + int(mm) * 60 + int(ss)

In [7]:
usedtime = data['USEDTIME']
reqtime = data['REQTIME']
usedtime_seconds = []
reqtime_seconds = []

for string in usedtime:
    usedtime_seconds.append(get_seconds(string))
    
for string in reqtime:
    reqtime_seconds.append(get_seconds(string))

In [10]:
data['usedtime_seconds'] = usedtime_seconds
data.head()

C:\Users\Pupnsuds90\AppData\Local\Temp\ipykernel_19772\2618691274.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['usedtime_seconds'] = usedtime_seconds


,JOBID,ACCOUNT,USER,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,GPUS,PARTITION,EXITCODE,STATE,usedtime_seconds
0,32880657,malanga,arline,18.66M,2-00:00:00,00:13:13,1,2,1,pascal,0:0,COMPLETED,793
2,32880718,glasshouse,brady,402.32M,05:00:00,00:08:57,1,3,1,maxwell,0:0,COMPLETED,537
3,32880719,glasshouse,brady,417.62M,20:00:00,05:09:46,1,1,1,pascal,0:0,COMPLETED,18586
4,32880720,glasshouse,brady,450.09M,15:00:00,04:11:10,1,1,1,pascal,0:0,COMPLETED,15070
5,32880721,glasshouse,brady,403.40M,3-12:00:00,18:49:36,1,1,1,pascal,0:0,COMPLETED,67776


In [13]:
data['USEDMEM'] = data['USEDMEM'].str.replace('M', '').astype('float')
data['USEDMEM']

C:\Users\Pupnsuds90\AppData\Local\Temp\ipykernel_19772\3372445554.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['USEDMEM'] = data['USEDMEM'].str.replace('M', '').astype('float')


0           18.66
2          402.32
3          417.62
4          450.09
5          403.40
           ...   
336933     137.96
336934    5093.70
336936     156.21
336942    2820.52
336948     145.24
Name: USEDMEM, Length: 277475, dtype: float64

In [15]:
data['weighted_usedmem'] = (data['USEDMEM'] * data['usedtime_seconds']) / data['usedtime_seconds'].sum()
data['weighted_usedmem']

C:\Users\Pupnsuds90\AppData\Local\Temp\ipykernel_19772\754710002.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['weighted_usedmem'] = (data['USEDMEM'] * data['usedtime_seconds']) / data['usedtime_seconds'].sum()


0         0.000009
2         0.000135
3         0.004867
4         0.004253
5         0.017143
            ...   
336933    0.000004
336934    0.006678
336936    0.000024
336942    0.001190
336948    0.000008
Name: weighted_usedmem, Length: 277475, dtype: float64

In [23]:
data['weighted_usedmem_by_GPU'] = data['weighted_usedmem'] / data['GPUS']

C:\Users\Pupnsuds90\AppData\Local\Temp\ipykernel_19772\1363503200.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['weighted_usedmem_by_GPU'] = data['weighted_usedmem'] / data['GPUS']


In [ ]:
maxwell_data = data[data['PARTITION'] == 'maxwell']

In [ ]:
maxwell_data